## Case Study: Solve recaptcha v2 with 2captcha service and requests

Created by [tanyongsheng.net](https://tanyongsheng.net)

Reference: 
1. How to Solve Captcha / ReCaptcha - Python and 2captcha https://www.youtube.com/watch?v=R6QddZzCOwM & https://github.com/eupendra/2captcha_demo/blob/main/demo_requests.py
2. Bypass captcha in Python: https://2captcha.com/lang/python

----

Prerequisite:
1. Buy 2Captcha credit at https://2captcha.com/?from=22013304 (Note: this is an affiliate link)

In [1]:
%pip install requests
%pip install lxml
%pip install 2captcha-python

In [2]:
login_page_url = "https://www.investingnote.com/users/sign_in"

In [6]:
import os
from dotenv import load_dotenv

_ = load_dotenv()


In [13]:
import requests
from lxml import html

# get crsf token
def get_csrf_token(session, url):
    response = session.get(url=url)
    tree = html.fromstring(response.content)
    csrf_token = tree.xpath("//input[@name='authenticity_token']/@value")[0]
    return csrf_token

In [14]:
from twocaptcha import TwoCaptcha

solver=TwoCaptcha(os.environ["2CAPTCHA_API_KEY"])
def solve_recaptcha(sitekey, url):
    # to solve recaptcha v2
    try:
        result = solver.recaptcha(sitekey=sitekey, 
                                  url=url)
    except Exception as e:
        exit(e)
    return result.get('code')


In [15]:
session = requests.Session()

headers = {"Content-Type": "application/x-www-form-urlencoded",
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
csrf_token = get_csrf_token(session=session, url=login_page_url)
recaptcha_response = solve_recaptcha(sitekey= os.environ["INVESTINGNOTE_SITE_KEY"], 
                                     url=login_page_url)

payload = {"utf8": "✓",
           "authenticity_token": csrf_token,
           "user[login]": os.environ["INVESTINGNOTE_USERNAME"], 
           "user[password]": os.environ["INVESTINGNOTE_PASSWORD"],
           "g-recaptcha-response": recaptcha_response, 
           "user[remember_me]": 0,
           "user[remember_me]": 1
           }

response = session.request("POST", 
    url="https://www.investingnote.com/users/sign_in",
    data=payload,
    headers=headers
    )
print(response.status_code)
print(response.url)
print("--------------------")
print(response.content)


200
https://www.investingnote.com/
--------------------
b'<html>\n<head>\n<title>\nInvestingNote.com - User pikachong1 Home Page\n</title>\n<meta content=\'The social network platform for sharing investing / trading ideas on stock markets. The next generation stock discussion forum.\' name=\'description\'>\n<meta content=\'The social network platform for sharing investing / trading ideas on stock markets. The next generation stock discussion forum.\' name=\'og:description\'>\n<meta content=\'InvestingNote.com - User pikachong1 Home Page\' name=\'og:title\'>\n<meta content=\'https://www.investingnote.com/assets/investingnote_og_image-16b12ccb23c99a575620b361df0228d04d6bc51dd8572bb94a6c03544cdf4a43.png\' name=\'og:image\'>\n<meta content=\'//www.investingnote.com\' name=\'og:url\'>\n<meta content=\'InvestingNote\' name=\'og:site_name\'>\n<meta content=\'stock,investor,Singapore,connect,network,investing,social,network,trade,trading\' name=\'keywords\'>\n<meta content=\'width=device-width